## Homework 4, Problem 4 Classification on real data

ECE C143A/C243A, Spring Quarter 2018, Prof. J.C. Kao, TAs T. Monsoor, X. Jiang and X. Yang

## Background
Neural prosthetic systems can be built based on classifying neural activity related to planning. As described in class, this is analogous to mapping patterns of neural activity to keys on a keyboard.
In this problem, we will apply the results of Problems 1 and 2 to real neural data. The neural data were recorded using a 100-electrode array in premotor cortex of a macaque monkey1. The dataset can be found on CCLE as `ps4_realdata.mat`.

The following describes the data format. The `.mat` file is loaded into Python as a dictionary with two keys: `train_trial` contains the training data and `test_trial` contains the test data. Each of these contains spike trains recorded simultaneously from 97 neurons while the monkey reached 91 times along each of 8 different reaching angles.

The spike train recorded from the $i_{th}$ neuron on the $n_{th}$ trial of the $k_{th}$ reaching angle is accessed as 

`data['train_trial'][n,k][1][i,:]`

where n = 0,...,90, k = 0,...,7, and i = 0, . . . , 96.  The [1] in between [n,k] and [i,:] does not mean anything for this assignment and is simply an "artifact" of how the data is structured. A spike train is represented as a sequence of zeros and ones, where time is discretized in 1 ms steps. A zero indicates that the neuron did not spike in the 1 ms bin, whereas a one indicates that the neuron spiked once in the 1 ms bin. The structure test trial has the same format as train trial.

Each spike train is 700 ms long (and thus represented by an array of length 700).  This comprises a 200ms baseline period (before the reach target turned on), a 500ms planning period (after the reach target turned on).  Because it takes time for information about the reach target to arrive in premotor cortex (due to the time required for action potentials to propagate and for visual processing), we will ignore the first 150ms of the planning period.  *** FOR THIS PROBLEM, we will take spike counts for each neuron within a single 200ms bin starting 150ms after the reach target turns on. ***

In other words, to calculate firing rates, you will calculate it over the 200ms window: 

`data['train_trial'][n,k][1][i,350:550]`

In [118]:
import numpy as np
import numpy.matlib as npm
import matplotlib.pyplot as plt
import scipy.special
import scipy.io as sio
import math

data = sio.loadmat('ps4_realdata.mat') # load the .mat file.
NumTrainData = data['train_trial'].shape[0]
NumClass = data['train_trial'].shape[1]
NumTestData = data['test_trial'].shape[0]

# Reloading any code written in external .py files.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### (a) (8 points) 
Fit the ML parameters of model i) to the training data (91 × 8 observations of a length 97 array of neuron firing rates). 

To calculate the firing rates, use a single 200ms bin starting from 150ms after the target turns on.  This corresponds to using `data['train_trial'][n,k][1][i, 350:550]` to calculate all firing rates.  This corresponds to a 200ms window that turns on 150ms after the reach turns on.

Then, use these parameters to classify the test data (91 × 8 data points) according to the decision rule (1). What is the percent of test data points correctly classified?

In [119]:
##4a

# Calculate the firing rates.

trainDataArr =  np.zeros((NumClass,NumTrainData,97)) # contains the firing rates for all neurons on all 8 x 91 trials in the training set
testDataArr =  np.zeros((NumClass,NumTestData,97)) # for the testing set.

for classIX in range(NumClass):
    for trainDataIX in range(NumTrainData):   
        trainDataArr[classIX,trainDataIX,:] = np.sum(data['train_trial'][trainDataIX,classIX][1][:,350:550],1)
    for testDataIX in range(NumTestData):        
        testDataArr[classIX,testDataIX,:]=np.sum(data['test_trial'][testDataIX,classIX][1][:,350:550],1)
#====================================================#
# YOUR CODE HERE:
#   Fit the ML parameters of model i) to training data
#====================================================#
modParam1 = {}
Total_number = trainDataArr[0].shape[0] + trainDataArr[1].shape[0] + trainDataArr[2].shape[0] + trainDataArr[3].shape[0] + trainDataArr[4].shape[0] + trainDataArr[5].shape[0] + trainDataArr[6].shape[0] + trainDataArr[7].shape[0]
modParam1['pi'] = np.array((1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8))
modParam1['mean'] = np.array((np.mean(trainDataArr[0], axis = 0), np.mean(trainDataArr[1], axis=0), np.mean(trainDataArr[2], axis=0), np.mean(trainDataArr[3], axis=0), np.mean(trainDataArr[4], axis=0), np.mean(trainDataArr[5], axis=0), np.mean(trainDataArr[6], axis=0), np.mean(trainDataArr[7], axis=0)))
modParam1['cov'] = np.array(trainDataArr[0].shape[0]/Total_number*np.cov(trainDataArr[0].T)) + np.array(trainDataArr[1].shape[0]/Total_number*np.cov(trainDataArr[1].T)) + np.array(trainDataArr[2].shape[0]/Total_number*np.cov(trainDataArr[2].T)) + np.array(trainDataArr[3].shape[0]/Total_number*np.cov(trainDataArr[3].T)) + np.array(trainDataArr[4].shape[0]/Total_number*np.cov(trainDataArr[4].T)) + np.array(trainDataArr[5].shape[0]/Total_number*np.cov(trainDataArr[5].T)) + np.array(trainDataArr[6].shape[0]/Total_number*np.cov(trainDataArr[6].T)) + np.array(trainDataArr[7].shape[0]/Total_number*np.cov(trainDataArr[7].T))
#====================================================#
# END YOUR CODE
#====================================================# 

#====================================================#
# YOUR CODE HERE:
#   Classify the test data and print the accuracy
#====================================================#
classification_correct = 0
def twoxmatmul(a,b,c):
    half = np.matmul(a,b)
    returnval = np.matmul(half,c)
    return returnval
for i in range(NumClass):
    for j in range(NumTestData):
        values = []
        for k in range(NumClass):
            val1 = np.transpose(np.matmul(np.linalg.inv(modParam1['cov']), modParam1['mean'][k]))
            val2 = (-1/2)*twoxmatmul(np.transpose(modParam1['mean'][k]), np.linalg.inv(modParam1['cov']), modParam1['mean'][k]) + np.log(modParam1['pi'][k])
            returnval = np.matmul(val1,testDataArr[i][j]) + val2
            values.append(returnval)
        maxval = np.argmax(values)
        #print(maxval)
        if maxval == i:
            classification_correct += 1


print(classification_correct/(91*8) * 100)
#====================================================#
# END YOUR CODE
#====================================================# 


96.01648351648352


#### Question:
What is the percent of test data points correctly classified?

#### Your answer: 
96.01648351648352

### (b) (6 points) 
Repeat part (a) for model ii). You `should encounter a Python error` when classifying the test data. What is this error? Why did the Python error occur? What would we need to do to correct this error?

To be concrete, the output of this cell should be a `Python error` and that's all fine.  But we want you to understand what the error is so we can fix it later.


In [120]:
##4b

#====================================================#
# YOUR CODE HERE:
# Fit the ML parameters of model ii) to training data
#====================================================#
modParam2 = {}
Total_number = trainDataArr[0].shape[0] + trainDataArr[1].shape[0] + trainDataArr[2].shape[0] + trainDataArr[3].shape[0] + trainDataArr[4].shape[0] + trainDataArr[5].shape[0] + trainDataArr[6].shape[0] + trainDataArr[7].shape[0]
modParam2['pi'] = np.array((1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8))
modParam2['mean'] = np.array((np.mean(trainDataArr[0], axis = 0), np.mean(trainDataArr[1], axis=0), np.mean(trainDataArr[2], axis=0), np.mean(trainDataArr[3], axis=0), np.mean(trainDataArr[4], axis=0), np.mean(trainDataArr[5], axis=0), np.mean(trainDataArr[6], axis=0), np.mean(trainDataArr[7], axis=0)))
cov1 = np.array(np.cov(trainDataArr[0].T))
cov2 = np.array(np.cov(trainDataArr[1].T))
cov3 = np.array(np.cov(trainDataArr[2].T))
cov4 = np.array(np.cov(trainDataArr[3].T))
cov5 = np.array(np.cov(trainDataArr[4].T))
cov6 = np.array(np.cov(trainDataArr[5].T))
cov7 = np.array(np.cov(trainDataArr[6].T))
cov8 = np.array(np.cov(trainDataArr[7].T))
covtotal = np.array((cov1, cov2, cov3, cov4, cov5, cov6, cov7, cov8))
modParam2['cov'] = covtotal


classification_correct = 0
def twoxmatmul(a,b,c):
    half = np.matmul(a,b)
    returnval = np.matmul(half,c)
    return returnval
for i in range(NumClass):
    for j in range(NumTestData):
        values = []
        for k in range(NumClass):
            val1 = np.transpose(np.matmul(modParam2['mean'][k], np.linalg.inv(modParam2['cov'][k])))
            val2 = (-1/2)*twoxmatmul(np.transpose(modParam2['mean'][k]), np.linalg.inv(modParam2['cov'][k]), modParam2['mean'][k]) + np.log(modParam2['pi'][k])
            val3 = np.linalg.inv(modParam2['cov'][k])
            returnval = (-1/2)*(np.matmul(val3,testDataArr[i][j])) + (np.matmul(val1,testDataArr[i][j]))+val2
            values.append(returnval)
        maxval = np.argmax(values)
        #print(maxval)
        if maxval == i:
            classification_correct += 1
print(classification_correct/(91*8) * 100)
#====================================================#
# END YOUR CODE
#====================================================# 


LinAlgError: Singular matrix

#### Question:
Why did the python error occur? What would we need to do to correct this error?

#### Your answer:  
We get this error because the sigma matrix is a singluar matrix meaning that it cannot be inverted. Hence, when we try to find the np.linalg.inv(modParam2['cov'][k]), it is not possible to do causing the error to be thrown. 

### (c) (8 points) 
Correct the problem from part (b) by detecting and then removing offending neurons that cause the error. Now, what is the percent of test data points correctly classified? Is it higher or lower than your answer to part (a)? Why might this be?

In [145]:
##4c
neuronsToRemove = []
#====================================================#
# YOUR CODE HERE:
#   Detect and then remove the offending neurons, so that 
#   you no longer run into the bug in part (b).
#====================================================#
remove = []
for i in range(NumClass):
    for j in range(97):
        if (not np.any(trainDataArr[i,:,j])):
            remove.append(j)
        
neuronsToRemove = list(set(remove))
#print(neuronsToRemove)
#====================================================#
# END YOUR CODE
#====================================================# 
##
#====================================================#
# YOUR CODE HERE:
# Fit the ML parameters,classify the test data and print the accuracy
#====================================================#
neuronsToKeep = list(range(0,97))
for i in neuronsToRemove:
    neuronsToKeep.remove(i)
newTrainData = trainDataArr[:,:,neuronsToKeep]
newTrainData = np.array(newTrainData)
newTestData = testDataArr[:,:,neuronsToKeep]
newTestData = np.array(newTestData)


modParam2 = {}
Total_number = newTrainData[0].shape[0] + newTrainData[1].shape[0] + newTrainData[2].shape[0] + newTrainData[3].shape[0] + newTrainData[4].shape[0] + newTrainData[5].shape[0] + newTrainData[6].shape[0] + newTrainData[7].shape[0]
modParam2['pi'] = np.array((1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8))
modParam2['mean'] = np.array((np.mean(newTrainData[0], axis = 0), np.mean(newTrainData[1], axis=0), np.mean(newTrainData[2], axis=0), np.mean(newTrainData[3], axis=0), np.mean(newTrainData[4], axis=0), np.mean(newTrainData[5], axis=0), np.mean(newTrainData[6], axis=0), np.mean(newTrainData[7], axis=0)))
cov1 = np.array(np.cov(newTrainData[0].T))
cov2 = np.array(np.cov(newTrainData[1].T))
cov3 = np.array(np.cov(newTrainData[2].T))
cov4 = np.array(np.cov(newTrainData[3].T))
cov5 = np.array(np.cov(newTrainData[4].T))
cov6 = np.array(np.cov(newTrainData[5].T))
cov7 = np.array(np.cov(newTrainData[6].T))
cov8 = np.array(np.cov(newTrainData[7].T))
covtotal = np.array((cov1, cov2, cov3, cov4, cov5, cov6, cov7, cov8))
modParam2['cov'] = covtotal


classification_correct = 0
def twoxmatmul(a,b,c):
    half = np.matmul(a,b)
    returnval = np.matmul(half,c)
    return returnval
for i in range(NumClass):
    for j in range(NumTestData):
        values = []
        for k in range(NumClass):
            val3 = np.linalg.inv(modParam2['cov'][k])
            val1 = np.transpose(np.matmul(modParam2['mean'][k], np.linalg.inv(modParam2['cov'][k]))) - (1/2*np.matmul(val3,newTestData[i,j]))
            val2 = (-1/2)*twoxmatmul(np.transpose(modParam2['mean'][k]), np.linalg.inv(modParam2['cov'][k]), modParam2['mean'][k]) + np.log(modParam2['pi'][k]) - (1/2*np.log(np.linalg.det(modParam2['cov'][k])))
            returnval = (np.matmul(val1,newTestData[i,j]))+val2
            values.append(returnval)
        maxval = np.argmax(values)
        if maxval == i:
            classification_correct += 1
print(classification_correct/(8*91) * 100)
#====================================================#
# END YOUR CODE
#====================================================# 

44.09340659340659


#### Question:
What is the percent of test data points correctly classified? Is it higher or lower than your answer to part (a)? Why might this be?

#### Your answer:  
The percentage of test data points correctly classified is 44.09%. This is lower than my answer to part a. This is because we have individial covariances for each class and we have removed some of the neurons from our data

### (d) (8 points) 
Now we classify using a naive Bayes model. Repeat part (a) for model iii). Keep the convention in part (c), where offending neurons were removed from the anal- ysis.

In [163]:
##4d
#====================================================#
# YOUR CODE HERE:
# Fit the ML parameters,classify the test data and print the accuracy
#====================================================#
modParam3 = {}
modParam3['pi'] = np.array((1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8))
modParam3['mean'] = np.array((np.mean(newTrainData[0], axis = 0), np.mean(newTrainData[1], axis=0), np.mean(newTrainData[2], axis=0), np.mean(newTrainData[3], axis=0), np.mean(newTrainData[4], axis=0), np.mean(newTrainData[5], axis=0), np.mean(newTrainData[6], axis=0), np.mean(newTrainData[7], axis=0)))

classification_correct = 0
def twoxmatmul(a,b,c):
    half = np.matmul(a,b)
    returnval = np.matmul(half,c)
    return returnval

for i in range(NumClass):
    for j in range(NumTestData):
        values = []
        for k in range(NumClass):
            vals = []
            for a in range(87):
                vals.append(newTestData[i,j,a]*np.log(modParam3['mean'][k][a]) - modParam3['mean'][k][a])
            values.append(np.log(modParam3['pi'][k]) + np.sum(vals))
        maxval = np.argmax(values)
        if maxval == i:
            classification_correct += 1
print(classification_correct/(8*91) * 100)
#====================================================#
# END YOUR CODE
#====================================================# 

92.03296703296702


#### Question:
what is the percent of test data points correctly classified? 

#### Your answer: 
The accuracy percentage is: 92.03296703296702